### View data
Notebook used for viewing data frame by frame

In [ ]:
import os 
import cv2
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
COLORS = {
    'vehicle' : (255, 0, 0),
    'truck' : (0, 255, 0),
    'motorcycle' : (0, 0, 255),
    'other' : (255, 255, 255)
}

In [ ]:
LABEL_FOLDER = './data/'
labels = pd.read_csv(os.path.join(LABEL_FOLDER, 'data.csv'))
labels = labels.sort_values('label')
labels['code'] = labels.label.apply(lambda x: x.split('_')[0])
prev_name = ''
prev_value = 1
for idx, row in labels.iterrows():
    if row.code == prev_name:
        prev_value += 1
    else:
        prev_value = 1
        prev_name = row.code
    code = '{}_{}'.format(prev_name, str(prev_value).zfill(2))
    labels.at[idx, 'code'] = code
labels = labels.set_index('code')
labels = labels.sort_index()
labels.fillna(0, inplace=True)
labels[['label']]

In [ ]:
def set_first_frame(df, fps):
    first_time = df.TimeStamp.astype(int).min() / 1e9
    if first_time > 0:
        first_frame = np.round(first_time * fps)
        #print ('First frame = {}'.format(first_frame))
        tmp = df.groupby('TimeStamp').size().sort_index().reset_index().drop(0, axis=1)
        tmp['frame'] = np.arange(len(tmp)) + first_frame
        df = df.drop('frame', axis=1).merge(tmp, on='TimeStamp')
    return df

def change_values(df):
    """
    Put everything that you want to change in data here.
    """
    df.type.replace(to_replace = 'motorcycle', vlaue='vehicle', inplace=True)
    return df

def read_data(idx):
    df = pd.read_excel(os.path.join(LABEL_FOLDER, labels.at[idx, 'label']))
    df.TimeStamp = pd.to_timedelta(df.TimeStamp)
    tmp = df.groupby('TimeStamp').size().sort_index().reset_index().drop(0, axis=1)
    tmp['frame'] = np.arange(len(tmp)) - labels.at[idx, 'shift']
    df = df.merge(tmp, on='TimeStamp')
    
    cap = cv2.VideoCapture(labels.loc[idx, 'video']) 
    fps = cap.get(cv2.CAP_PROP_FPS)
    df = set_first_frame(df, fps)
    df.frame += labels.at[idx, 'shift']
    df = df[df.frame > 0]
    for col in ['x', 'y', 'frame']:
        df[col] = df[col].astype(int)
    return df, cap


In [ ]:
def add_rectangles(image, df):
    for index, row in df.iterrows():
        if row.type in COLORS:
            color = COLORS[row.type]
        else:
            color = COLORS['other']
        cv2.rectangle(image, (int (row.x), int(row.y)), (int (row.x + row.width), int (row.y + row.height)), color, 3)
    return image

def display_frame(cap, df, frameno = 1):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frameno)
    ret, image = cap.read()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = add_rectangles(image, df[df.frame == frameno])

    plt.figure(figsize=(12, 12))
    plt.imshow (image)
    plt.title('Frame no. {}'.format(frameno))
    return image

def display_frames(cap, df, frame_start = None, frame_end = None):
    if frame_start is None:
        frame_start = df.frame.min()
    if frame_end is None:
        frame_end = df.frame.max()
    frameno = frame_start
    cap.set(cv2.CAP_PROP_POS_FRAMES, frameno)
    while (frameno < frame_end):
        ret, image = cap.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = add_rectangles(image, df[df.frame == frameno])

        plt.figure(figsize=(12, 12))
        plt.imshow (image)
        plt.title('Frame no. {}'.format(frameno))
        plt.show()
        frameno += 1
        print (frameno)
    


In [ ]:
idx = labels.index[-1]
print ('Reading {}.'.format(idx))
df, cap = read_data(idx)
#df.frame = df.frame -3
df = df[df.frame > 0]
df.head()

In [ ]:
image1 = display_frame(cap, df, df.frame.min())

### Display all frames

In [ ]:
display_frames(cap, df)